<a href="https://colab.research.google.com/github/yanncoadou/MLtutorials/blob/ESIPAP2023/DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Machine learning hands-on</center></h1>
<h2><center>Deep neural networks</center></h2>

# Classifier zoo

Play with various algorithms as implemented in scikit-learn and tensorflow.

In [ ]:
# Add classifiers into a list to access them later
classifiers = []

## Comparison

In [ ]:
#my_roc_curves(classifiers, X_test, y_test)

## Other classifiers
We have only seen tree-based classifiers from scikit-learn above. Tree-based algorithms tend to be fast and to work decently out-of-the-box.

There are many more types of classifiers:

1.   implemented in scikit-learn: see the [user's guide](https://scikit-learn.org/stable/user_guide.html) for A LOT of different algorithms
2.   in various other packages:
- for decision trees (more powerful than scikit-learn implementations): [XGBoost](https://xgboost.readthedocs.io/en/stable/), [LightGBM](https://lightgbm.readthedocs.io/en/latest/), [CatBoost](https://catboost.ai/)

- for neural networks: [TensorFlow](https://www.tensorflow.org/), [PyTorch](https://pytorch.org/)


### Neural networks
It is possible to use neural networks from scikit-learn (`from sklearn.neural_network import MLPClassifier`). In the following we will use more advanced implementations with [Tensorflow](https://www.tensorflow.org).

In [ ]:
## Note: there is no predict_proba in Keras, predict returns the NN output
#print("predict: \n",model.predict(X_test[:5]))
## "round" to transform NN output into class 0 or 1
#print("Accuracy: ", accuracy_score(y_test, model.predict(X_test).round()))

# High energy physics application

### Standard imports and useful functions
 (from top of notebook)

In [ ]:
# scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, make_circles
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve, accuracy_score, roc_auc_score, roc_curve, RocCurveDisplay

%matplotlib inline
import seaborn as sns # seaborn for nice plots
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
np.random.seed(31415) # set the np random seed for reproducibility

Function to plot ROC curves

In [ ]:
def my_roc_curves(models, X_test, y_test, weights_test=np.array([])):
  if weights_test.size == 0:
    weights_test = np.ones(len(X_test))
  for i, clf in enumerate(models):
    if hasattr(clf, "predict_proba"):
      y_preds_clf = clf.predict_proba(X_test)[:,1].reshape(-1)
    else:
      y_preds_clf = clf.predict(X_test)
    fpr_clf,tpr_clf,_ = roc_curve(y_true=y_test, y_score=y_preds_clf, sample_weight=weights_test)
    auc_test_clf = roc_auc_score(y_true=y_test, y_score=y_preds_clf, sample_weight=weights_test)
    plt.plot(fpr_clf, tpr_clf, label='{} (AUC  = {})'.format(clf.__class__.__name__,np.round(auc_test_clf,decimals=2)))
  plt.plot([0, 1], [0, 1], color='black', linestyle='--')
  plt.xlim([-0.05, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver Operating Characteristic')
  plt.legend(loc="lower right");

## Input dataset

Data created from ATLAS Open Data by David Rousseau. See doc:

http://opendata.atlas.cern/release/2020/documentation/datasets/intro.html

### Downloading dataset

In [ ]:
try:
  import google.colab
  COLAB = True # if running in COLAB
  print("You are running on Colab.")
except:
  COLAB = False # if not running on COLAB
  print("You are NOT running on Colab, you need to fix the data file path below.")
import tensorflow as tf

In [ ]:
if COLAB:
  #### Reading files from Google Drive
  # Need a Google account to be identified
  #!pip install PyDrive
  import os
  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive
  from google.colab import auth
  from oauth2client.client import GoogleCredentials
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

In [ ]:
import os
datapath=""
if not os.path.isfile("dataWW_d1_600k.csv.gz"):
  if COLAB:
    #attach dataset from google drive 
    download = drive.CreateFile({'id': '1nlXp7P-xq_jip4aPE0j0mnPhYnIOcBv4'})
    download.GetContentFile("dataWW_d1_600k.csv.gz")
    !ls -lrt
  else :
    # Make sure the file is available locally. 
    # Should be downloaded from https://drive.google.com/uc?id=1nlXp7P-xq_jip4aPE0j0mnPhYnIOcBv4
    !ls -lrt # what is in the local directory
    datapath="/directory/where/you/stored/dataWW_d1_600k.csv.gz"
    !ls -lrt {datapath} # what is in the data directory
    datapath=os.path.abspath(datapath).replace("\ ", " ")  # try to normalise the path (annoyance with the space)
    print ("Will take data from : ",datapath)
filename=os.path.join(datapath,"dataWW_d1_600k.csv.gz")
print("File to be loaded: ")
!ls -l {filename}

# Loading dataset
dfall = pd.read_csv(filename) 
print ("\nFile loaded with ",dfall.shape[0], " events ")


You should now see:

`File loaded with  600000  events`

### Reading ROOT files
*Only informative, not used afterwards*

Input files in HEP often in ROOT format, not understandable by ML packages. You can convert your ROOT files to CSV or HDF5, or read them directly with [uproot](https://uproot.readthedocs.io) without installing ROOT.


In [ ]:
import os
filename="dataWW_d1_600k.root"
# Make sure the file is available locally. 
if not os.path.isfile(filename):
  !wget https://www.cppm.in2p3.fr/~coadou/Work/Classes/AdvancedStatsHEP_IPhU2022/{filename}
!ls -rtl

In [ ]:
!pip install uproot awkward
import uproot

In [ ]:
#file = uproot.open(filename)
#file.keys()
tree = uproot.open(filename+":myTree")
tree.keys()
#tree.typenames()

In [ ]:
rootAA = tree.arrays() # AwkwardArray
rootpd = tree.arrays(library='pd') # pandas array
rootnp = tree.arrays(library='np') # numpy array

In [ ]:
display(rootAA)
display(rootpd)
display(rootnp)

In [ ]:
# get only specific branches 
rootpd = tree.arrays(['lep_pt_0','lep_pt_1','met_et'], library='pd')
rootpd.head()

In [ ]:
# create new variable
rootpd = tree.arrays(['lep_pt_0','lep_pt_1','met_et', 'dphi'], aliases={"dphi": "lep_phi_1-lep_phi_0"}, library='pd')
rootpd.head()

Then you can use your array as the one obtained from the CSV file.

### Checking the content

In [ ]:
#dumping list of features
dfall.columns

In [ ]:
#examining first few events
display(dfall.head())

In [ ]:
#examining feature distributions
dfall.describe()

***Event weights***

In [ ]:
label_nevents = (dfall[dfall.label==0].shape[0], dfall[dfall.label==1].shape[0] )
print("Number of events per class (B, S):",label_nevents)

label_weights = (dfall[dfall.label==0].mcWeight.sum(), dfall[dfall.label==1].mcWeight.sum() ) 
print("Total weight per class (B, S):    ",label_weights)

## Event selection

Only keep events with exactly two leptons for this exercise.

Only keep events with positive weight, as many ML tools choke on negative weight.

*Note: This is in principle WRONG, only possibly valid if your positive and negative weight events are statistically similar (could then also take the absolute value of the weight to increase statistics).*


In [ ]:
print ("Df shape before selection:", dfall.shape)

fulldata=dfall[ (dfall.lep_n==2) & (dfall.mcWeight > 0)]  

print ("Df shape after selection: ",fulldata.shape)

In [ ]:
# Hide label and weights in separate vectors (not discriminating features)
# WARNING : there should be neither selection nor shuffling later on! (otherwise misalignement)
target = fulldata["label"]
weights = fulldata["mcWeight"]

# for simplicity only keep some features
# this is actually making a deep copy from fulldata
data=pd.DataFrame(fulldata, columns=["met_et","met_phi","lep_pt_0","lep_pt_1",'lep_phi_0', 'lep_phi_1'])
#data=pd.DataFrame(fulldata, columns=["met_et","met_phi","lep_pt_0","lep_pt_1",'lep_eta_0', 'lep_eta_1', 'lep_phi_0', 'lep_phi_1','jet_n','jet_pt_0',
#       'jet_pt_1', 'jet_eta_0', 'jet_eta_1', 'jet_phi_0', 'jet_phi_1']

print ("Df shape of dataset to be used:",data.shape)

### Event weights

In [ ]:
fig,ax=plt.subplots()

bins=np.linspace(-1,1,101)
plt.hist(weights[target==0]*1000,bins=bins,color='b',alpha=0.5,density=True,label='B ackground')
plt.hist(weights[target==1]*10000,bins=bins,color='r',alpha=0.5,density=True,label='S ignal (Wx10)')
plt.legend(loc='best')
ax.set_xlabel('weight*1000')
plt.show()

### Feature engineering

Add more complex variables to the dataset.

In [ ]:
data["lep_deltaphi"]=np.abs(np.mod(data.lep_phi_1-data.lep_phi_0+3*np.pi,2*np.pi)-np.pi)

print(data.shape)
display(data.head())

### Plotting variables

In [ ]:
fig,ax=plt.subplots(1, 2, figsize=(12, 5))
data['met_et'].plot.hist(title='Missing Transverse Energy', log=True, ax=ax[0])
data[data.lep_pt_0+data.lep_pt_1>1000]['met_et'].plot.hist(bins=np.linspace(0,400,50),title='Missing Transverse Energy for large lepton Pt', ax=ax[1]);

In [ ]:
ax=data[target==0].plot.scatter(x='met_et', y='lep_pt_0',color="b",label="B")
data[target==1].plot.scatter(x='met_et', y='lep_pt_0',color="r",label="S",alpha=.5,ax=ax);

In [ ]:
data[data.lep_pt_0+data.lep_pt_1>2000].head()

In [ ]:
ax=data[target==0].hist(weights=weights[target==0],figsize=(15,12),bins=50,color='b',alpha=0.5,density=True,label="B")
ax=ax.flatten()[:data.shape[1]] # to avoid error if holes in the grid of plots (like if 7 or 8 features)
data[target==1].hist(weights=weights[target==1],figsize=(15,12),bins=50,color='r',alpha=0.5,density=True,ax=ax,label="S");


### Features correlation matrix

In [ ]:
fig,ax=plt.subplots(1, 2, figsize=(12, 5))

corrMatrix = data[target==0].corr()
ax[0].set_title("Background features correlation matrix")
sns.heatmap(corrMatrix.round(3), ax=ax[0], annot=True);

corrMatrix = data[target==1].corr()
ax[1].set_title("Signal features correlation matrix")
sns.heatmap(corrMatrix.round(3), ax=ax[1], annot=True);


## Sample splitting

In [ ]:
np.random.seed(31415) # set the random seed (used for the train/test splitting)

from sklearn.model_selection import train_test_split
train_size = 0.75 # fraction of sample used for training
val_size = 0.2 # fraction of training sample used for validation

# split only train/test
#X_train, X_test, y_train, y_test, weights_train, weights_test = \
#    train_test_split(data, target, weights, train_size=train_size)

#split in train/validation/test
X_holdout, X_test, y_holdout, y_test, weights_holdout, weights_test = \
    train_test_split(data, target, weights, train_size=train_size)
X_train, X_val, y_train, y_val, weights_train, weights_val = \
    train_test_split(X_holdout, y_holdout, weights_holdout, train_size=1-val_size)

print("Training sample:  ", X_train.shape)
print("Validation sample:", X_val.shape)
print("Testing sample:   ", X_test.shape)

In [ ]:
# Weight handling
class_weights_train = (weights_train[y_train == 0].sum(), weights_train[y_train == 1].sum())
print ("class_weights_train (B, S):",class_weights_train)

for i in range(len(class_weights_train)): # could have more than two classes
    weights_train[y_train == i] *= max(class_weights_train)/ class_weights_train[i] #equalize number of background and signal event
    weights_test[y_test == i] *= 1/(1-train_size) # increase test weight to compensate for sampling
    weights_val[y_val == i] *= 1/val_size/train_size # increase val weight to compensate for samplings
    
print ("Test:  total weight sig", weights_test[y_test == 1].sum())
print ("Test:  total weight bkg", weights_test[y_test == 0].sum())
print ("Train: total weight sig", weights_train[y_train == 1].sum())
print ("Train: total weight bkg", weights_train[y_train == 0].sum())
print ("Val:   total weight sig", weights_val[y_val == 1].sum())
print ("Val:   total weight bkg", weights_val[y_val == 0].sum())


## Network training

In [ ]:
try:
  import tensorflow as tf
except ImportError as e:
  !pip install tensorflow
  import tensorflow as tf
print (tf.__version__)  # preinstalled version 2.9.2 20230127
from tensorflow import keras

In [ ]:
tf.random.set_seed(1234) # to have reproducible networks
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)), # 1st hidden layer
  #tf.keras.layers.Dense(128, activation='relu'), # 2nd hidden layer
  tf.keras.layers.Dense(1,activation="sigmoid") # output layer
])

model.compile(loss="binary_crossentropy",
              optimizer="adam",
              #metrics=['accuracy', keras.metrics.AUC(name="auc")]) # if not using event weights
              weighted_metrics=['accuracy', keras.metrics.AUC(name="auc")])

In [ ]:
# notice "history" returned from fit
history = model.fit(X_train, y_train.values,
                    epochs=1,
                    #validation_split=0.2,                       # to be used with train/test split
                    validation_data=(X_val, y_val, weights_val), # to be used with train/val/test split
                    batch_size=1024,
                    sample_weight=weights_train.values, # if using weights
                    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

### Standardisation of inputs
Checking the impact on neural network training

In [ ]:
from sklearn.preprocessing import StandardScaler

print("Original mean and variance:")
for feature, mean, std in zip(data.columns,X_train.mean(0), X_train.std(0)):
  print("{:9}: {:7.4f} +/- {:7.4f}".format(feature,mean,std))

# Standardize features by removing the mean and scaling to unit variance
# in training sample
scaler = StandardScaler()
# ".values[:]" to keep dataframe and not convert to numpy array
X_train.values[:] = scaler.fit_transform(X_train)
# apply to testing/validation sample the transformation calculated on training sample
X_test.values[:] = scaler.transform(X_test)
X_val.values[:] = scaler.transform(X_val)

print("\nStandardised mean and variance:")
for feature, mean, std in zip(data.columns,X_train.mean(0), X_train.std(0)):
  print("{:9}: {:7.4f} +/- {:7.4f}".format(feature,mean,std))

In [ ]:
tf.random.set_seed(1234) # to have reproducible networks
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)), # 1st hidden layer
  #tf.keras.layers.Dense(128, activation='relu'), # 2nd hidden layer
  tf.keras.layers.Dense(1,activation="sigmoid") # output layer
])

model.compile(loss="binary_crossentropy",
              optimizer="adam",
              #metrics=['accuracy', keras.metrics.AUC(name="auc")]) # if not using event weights
              weighted_metrics=['accuracy', keras.metrics.AUC(name="auc")])

history = model.fit(X_train, y_train.values,
                    epochs=100,
                    #validation_split=0.2,   # to be used with train/test split
                    validation_data=(X_val, y_val, weights_val),
                    batch_size=1024,
                    sample_weight=weights_train.values,
                    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

classifiers.append(model)

y_pred_model = model.predict(X_test).reshape(-1)

*Compare the training loss/accuracy/AUC after epoch 1 with that obtained before standardisation.*

In [ ]:
density=True   # normalised to 1 (=> probability density function)
#density=False   # normalised to one year at LHC

plt.hist(y_pred_model[y_test == 0],
         color='b', alpha=0.5, 
         bins=30, range=[0,1],
         histtype='stepfilled', density=density,
         label='B (test)', weights=weights_test[y_test == 0])
plt.hist(y_pred_model[y_test == 1],
         color='r', alpha=0.5,
         bins=30, range=[0,1],
         histtype='stepfilled', density=density,
         label='S (test)', weights=weights_test[y_test == 1])

# also plot training sample output for comparison
# WARNING: not useful to diagnose overtraining!
y_train_model = model.predict(X_train).reshape(-1)
plt.hist(y_train_model[y_train == 0],
         color='b', alpha=0.5,  linewidth=2,
         bins=30, range=[0,1],
         histtype='step', density=density,
         label='B (train)', weights=weights_train[y_train == 0])
plt.hist(y_train_model[y_train == 1],
         color='r', alpha=0.5, linewidth=2,
         bins=30, range=[0,1],
         histtype='step', density=density,
         label='S (train)', weights=weights_train[y_train == 1])
plt.legend()
plt.title("NN model score");

In [ ]:
my_roc_curves([model], X_test, y_test, weights_test)

### Training monitoring

In [ ]:
fig,ax=plt.subplots(1, 2, figsize=(12, 5))
ax[0].plot(history.history['loss'],label="Training loss")
ax[0].plot(history.history['val_loss'],label="Validation loss")
ax[0].set_xlabel("Epoch")
ax[0].legend(loc='best');

ax[1].plot(history.history['auc'],label="Training AUC")
ax[1].plot(history.history['val_auc'],label="Validation AUC")
ax[1].set_xlabel("Epoch")
ax[1].legend(loc='best');

### Model saving

***Whole-model saving & loading***

You can save an entire Keras model to a directory. It will include:
- the model's architecture/config
- the model's weight values (which were learned during training)
- the model's compilation information (if `compile()` was called)
- the optimizer and its state, if any (this enables you to restart training where you left off)


In [ ]:
model.save("NNmodel")
!ls -a NNmodel/*

In [ ]:
print("Prediction from original model:")
display(model.predict(X_test[:5]))

reloaded_model=keras.models.load_model("NNmodel")
print("Prediction from reloaded model:")
display(reloaded_model.predict(X_test[:5]))

# full test
try:
  np.testing.assert_allclose(
    model.predict(X_test), reloaded_model.predict(X_test)
  )
  print("Predictions from original and reloaded models are identical")
except AssertionError:
  print("Error: predictions from original and reloaded models are different")

# further training
reloaded_model.fit(X_train, y_train.values,
                   epochs=5,
                   #validation_split=0.2,   # to be used with train/test split
                   validation_data=(X_val, y_val, weights_val),
                   batch_size=1024,
                   sample_weight=weights_train.values,
                   callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

*Compare first epoch values with original training.*

***Partial save***

Save a single HDF5 file containing the model's architecture, weights values, and `compile()` information.

Not saved (to be provided separately to resume training):
- external losses & metrics added via `model.add_loss()` & `model.add_metric()`
- computation graph of custom objects

In [ ]:
model.save("NNmodel.h5")
!ls -lrt --color
print("\nPrediction from original model:")
display(model.predict(X_test[:5]))

reloaded_model=keras.models.load_model("NNmodel.h5")
print("Prediction from reloaded model:")
display(reloaded_model.predict(X_test[:5]))

***Saving the architecture and weights***

Keeping the model's configuration and training weights in separate files

In [ ]:
arch = model.to_json()
with open('NNmodel.json', 'w') as arch_file:
  arch_file.write(arch)
model.save_weights('NNmodel_weights.h5')
!ls -lrt --color

In [ ]:
!python -m json.tool NNmodel.json

In [ ]:
with open('NNmodel.json', 'r') as f:
  reloaded_model = keras.models.model_from_json(f.read())
reloaded_model.summary()

reloaded_model.load_weights("NNmodel_weights.h5")
reloaded_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       #metrics=['accuracy', keras.metrics.AUC(name="auc")]) # if not using event weights
                       weighted_metrics=['accuracy', keras.metrics.AUC(name="auc")])
reloaded_model.fit(X_train, y_train.values,
                    epochs=1,
                    #validation_split=0.2,   # to be used with train/test split
                    validation_data=(X_val, y_val, weights_val),
                    batch_size=1024,
                    sample_weight=weights_train.values,
                    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])


### Model conversion to ONNX
Can be useful as ONNX provides common description to models coming from scikit-learn, tensorflow, pytorch, etc., and inference interface in python, C++, C#, etc. See [onnxruntime](https://onnxruntime.ai) documentation.

In [ ]:
!pip install tf2onnx
import tf2onnx

In [ ]:
#convert to ONNX
onnx_model = tf2onnx.convert.from_keras(model)[0]

In [ ]:
import onnx
onnx.save(onnx_model,'onnx_model.onnx')
!ls -rtl

In [ ]:
!pip install onnxruntime
import onnxruntime as ort
sess = ort.InferenceSession('onnx_model.onnx', providers = ['CPUExecutionProvider'])

In [ ]:
input_name = sess.get_inputs()[0].name
output_names = [n.name for n in onnx_model.graph.output]
print(input_name)
print(output_names)

In [ ]:
pred_onnx = sess.run(output_names,{input_name: X_test.values.astype(np.float32)})[0]
print(pred_onnx[:5].reshape(-1))
print(y_pred_model[:5])

In [ ]:
try:
  np.testing.assert_allclose(
    y_pred_model, pred_onnx.reshape(-1),
    rtol=1e-3, atol=0
  )
  print("Predictions from original and ONNX models are identical")
except AssertionError:
  print("Error: predictions from original and ONNX models are different")


## Boosted tree training

In [ ]:
# preinstalled version 0.9.0 20230127
!pip install xgboost --upgrade # install 1.7.3 20230127

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score # for binary classification if x > 0.5 -> 1 else -> 0
xgb = XGBClassifier(tree_method="hist")
xgb.fit(X_train, y_train.values,
        sample_weight=weights_train.values) # note that XGB 1.3.X requires positive weight
classifiers.append(xgb)

In [ ]:
my_roc_curves(classifiers, X_test, y_test, weights_test)

## Physics performance

### Permutation importance

A better way to show the importance of each feature is Permutation Importance, where each feature in turn is replaced by an instance of an other event (effectively switching it off by randomising).

Works on any classifier, not just DT-based. Can be estimated on any sample, not just training set.

However, report can be misleading in case of highly correlated variables.

Available in [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html).
   


In [ ]:
from sklearn.inspection import permutation_importance
importances = []
for clf in classifiers:
  result = permutation_importance(clf, X_test, y_test, n_repeats=1, random_state=42, n_jobs=2)
  importances.append(pd.Series(result.importances_mean, index=list(data.columns.values)))
  

In [ ]:
fig,ax=plt.subplots(2, 2, figsize=(18, 10))
for i, (clf,importance) in enumerate(zip(classifiers,importances), start=0):
  importance.plot.bar(ax = ax[i//2,i%2], subplots=True)
  ax[i//2,i%2].tick_params(labelrotation=0)
  ax[i//2,i%2].set_title("{} feature importance".format(clf.__class__.__name__))

Another implementation targetting HEP:

https://github.com/aghoshpub/permutationImportancePhysics 

In particular it allows to : 
   * use event weights
   * display directly the loss in whatever criterion (ROC auc, asimov significance) when the feature is switched off
   * display the feature importance for a specific subset (for example the most signal like)
   * it can even display which feature has the largest impact on systematics


In [ ]:
!pip install PermutationImportancePhysics
from permutationimportancephysics.PermutationImportance import PermulationImportance # note the delibrate typo PermuLation
#XGBoost
PI_xgb = PermulationImportance(model=xgb, X=X_test.values,y=y_test,weights=weights_test,\
                       n_iterations=1,usePredict_poba=True, scoreFunction="amsasimov", colNames=list(data.columns.values))
#PI_xgb.dislayResults()
plott_xgb = PI_xgb.plotBars()

#LightGBM    
PI_model = PermulationImportance(model=model, X=X_test.values,y=y_test,weights=weights_test,\
                         n_iterations=1,usePredict_poba=True, scoreFunction="amsasimov", colNames=list(data.columns.values))
#PI_gbm.dislayResults()
plott_model = PI_model.plotBars()

### Significance

Asimov significance (from [arXiv:1007.1727](https://arxiv.org/abs/1007.1727) eq. 97):

> AMS = $\sqrt{2\left((s+b)\ln\left(1+\frac{s}{b}\right) - s\right)} = \frac{s}{\sqrt{b}}\left(1+\mathcal{O}(s/b)\right)$

In [ ]:
from math import sqrt, log
def amsasimov(s,b):
  if b<=0 or s<=0:
      return 0
  try:
      return sqrt(2*((s+b)*log(1+float(s)/b)-s))
  except ValueError:
      print(1+float(s)/b)
      print (2*((s+b)*log(1+float(s)/b)-s))

In [ ]:
vamsasimovs = []
for clf in classifiers:
  y_pred_clf = clf.predict_proba(X_test)[:,1].reshape(-1)
  int_pred_test_sig_clf = [weights_test[(y_test ==1) & (y_pred_clf > th_cut)].sum() for th_cut in np.linspace(0,1,num=50)]
  int_pred_test_bkg_clf = [weights_test[(y_test ==0) & (y_pred_clf > th_cut)].sum() for th_cut in np.linspace(0,1,num=50)]

  vamsasimov_clf = [amsasimov(sumsig,sumbkg) for (sumsig,sumbkg) in zip(int_pred_test_sig_clf,int_pred_test_bkg_clf)]
  print("Z({}): {}".format(clf.__class__.__name__,np.round(max(vamsasimov_clf),decimals=3)))
  vamsasimovs.append(vamsasimov_clf)

In [ ]:
for i, (clf,vamsasimov_clf) in enumerate(zip(classifiers,vamsasimovs), start=0):
  plt.plot(np.linspace(0,1,num=50),vamsasimov_clf, label='AMS (Z_max({}) = {})'.format(clf.__class__.__name__,np.round(max(vamsasimov_clf),decimals=3)))
plt.title("Significance")
plt.xlabel("Threshold")
plt.ylabel("Significance")
plt.legend()
#plt.savefig("Significance.pdf")
plt.show()

### Hyperparameter optimisation
Can be done by hand, with [random search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) or [grid search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

Also dedicated packages doing Gaussian process optimisation or 'tree of Parzen estimators' (TPE) (e.g. [hyperopt](https://github.com/hyperopt/hyperop) or [optuna](https://optuna.org/)).

In [ ]:
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV

# specify parameters and distributions to sample from
param_dist_XGB = {'n_estimators': stats.randint(10, 500), #default 100
                  'learning_rate': stats.uniform(0.01, 0.5)} #def 0.3 
                  #'max_depth': stats.randint(3, 12)} # default 6

# default CV is 5 fold, reduce to 2 for speed concern
# default n_iter is 10 sets of parameters, reduce to 5 for speed concern
gsearch = RandomizedSearchCV(estimator = XGBClassifier(tree_method="hist",use_label_encoder=False,eval_metric='logloss'), 
                             param_distributions = param_dist_XGB, 
                             scoring='roc_auc',n_iter=5,cv=2,verbose=2)
gsearch.fit(X_train,y_train, sample_weight=weights_train);

In [ ]:
print ("Best parameters: ",gsearch.best_params_)
print ("Best score (on train dataset CV): ",gsearch.best_score_)
# Best model directly accessible if refit=True (default)
y_pred_gs = gsearch.predict_proba(X_test)[:,1]
print("... corresponding score on test dataset: ",roc_auc_score(y_true=y_test, y_score=y_pred_gs))

dfsearch=pd.DataFrame.from_dict(gsearch.cv_results_)
display(dfsearch.head())

fig,ax=plt.subplots(1, 3, figsize=(15, 5))
dfsearch.plot("param_n_estimators","mean_test_score",yerr="std_test_score",linestyle = 'None',marker="o", ax=ax[0])
ax[0].scatter(gsearch.best_params_['n_estimators'],gsearch.best_score_,color='red',marker="*",s=100,zorder=5)
dfsearch.plot("param_learning_rate","mean_test_score",yerr="std_test_score",linestyle = 'None',marker="o", ax=ax[1])
ax[1].scatter(gsearch.best_params_['learning_rate'],gsearch.best_score_,color='red',marker="*",s=100,zorder=5)
#dfsearch.plot("param_max_depth","mean_test_score",yerr="std_test_score",linestyle = 'None',marker="o", ax=ax[2])
#ax[2].scatter(gsearch.best_params_['max_depth'],gsearch.best_score_,color='red',marker="*",s=100,zorder=5);

# Your turn
Try and optimise a classifier on this dataset. You can play with:
*   type of classifier
*   hyperparameters of classifier
*   input variables
*   figure of merit

Remember that some algorithms are (much) more time consuming than others, and therefore take more time to optimise.

You can optimise by hand, or using some of the tools presented above.


In [ ]:
#your code goes here